In [1]:
nevecs = 50
ncomps = 500
nbasecomps = 25
crop_size=180
ncauses = 2
dr_method = 'PCA'

In [2]:
from pathlib import Path
import os, sys
import torchvision.transforms.functional as F
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [3]:

macaw_path = Path(os.getcwd())
sys.path.append(str(macaw_path) +'/')

exp_name = 'moin_bias'

train_csv_path = macaw_path/'splits2/exp199/train.csv'
train_img_path = macaw_path/exp_name/'train'

val_csv_path = macaw_path/'splits2/exp199/val.csv'
val_img_path = macaw_path/exp_name/'val'

pca_path = macaw_path/exp_name/f'train_hc_pca_{ncomps}.pkl'

train_path = macaw_path/exp_name/f'train_hc_data_{dr_method}_{ncomps}.pkl'
val_path = macaw_path/exp_name/f'val_hc_data_{dr_method}_{ncomps}.pkl'
model_base_path = macaw_path/'models'/exp_name/f'{dr_method}_{ncomps}'/f'{nevecs}'

if not os.path.exists(model_base_path):
    os.makedirs(model_base_path)

In [4]:
import pickle

with open(train_path, 'rb') as f:
    train = pickle.load(f)
    
disease_t = train['disease']
bias_t = train['bias']

encoded_t = train['encoded_data']
pca_t = train['pca']

In [5]:
with open(val_path, 'rb') as f:
    val = pickle.load(f)
    
disease_v = val['disease']
bias_v = val['bias']

encoded_v = val['encoded_data']
pca_v = val['pca']

## Causal Graph

In [6]:
disease_to_latents = [(0,i) for i in range(ncauses,nevecs+ncauses)]
bias_to_latents = [(1,i) for i in range(ncauses,nevecs+ncauses)]

autoregressive_latents = [(i,j) for i in range(ncauses,nevecs+ncauses) for j in range(i+1,nevecs+ncauses)]
edges = disease_to_latents + bias_to_latents + autoregressive_latents

## Priors

In [7]:
P_bias = np.sum(bias_t)/len(bias_t)
print(P_bias)

P_dis = np.sum(disease_t)/len(disease_t)
print(P_dis)

In [8]:
import torch
import yaml
from utils.helpers import dict2namespace

with open(macaw_path/'config'/'ukbb.yaml', 'r') as f:
    config_raw = yaml.load(f, Loader=yaml.FullLoader)
    
config = dict2namespace(config_raw)
config.device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [9]:
config.device

In [10]:
import torch.distributions as td

priors = [(slice(0,1),td.Bernoulli(torch.tensor([P_dis]).to(torch.device(config.device)))), # disease
          (slice(1,2),td.Bernoulli(torch.tensor([P_bias]).to(torch.device(config.device)))), # bias
          (slice(2,nbasecomps+2),td.Normal(torch.zeros(nbasecomps).to(config.device), torch.ones(nbasecomps).to(config.device))), # base_comps
          (slice(nbasecomps+2,nevecs+2),td.Normal(torch.zeros(nevecs-nbasecomps).to(config.device), torch.ones(nevecs-nbasecomps).to(config.device))), # new_comps
         ]

In [11]:
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, transforms

# Writer will output to ./runs/ directory by default
writer = SummaryWriter()

In [12]:
loss_vals = []

In [13]:
from macaw import MACAW

for e in range(0,ncomps-nbasecomps-1,nevecs-nbasecomps):
    save_path = model_base_path/f'{e}.pt'    
    ed_t = encoded_t[:,e:e+nevecs]
    ed_v = encoded_v[:,e:e+nevecs]
    print(e,e+nevecs)

    # if os.path.exists(save_path): 

    X = np.hstack([disease_t[:,np.newaxis], bias_t[:,np.newaxis], ed_t])   
    Y = np.hstack([disease_v[:,np.newaxis], bias_v[:,np.newaxis], ed_v])   

    macaw = MACAW.MACAW(config)
    loss_vals.append(macaw.fit_with_priors(X, Y, edges, priors, e))

    torch.save(macaw,save_path)

In [14]:
len(loss_vals)

In [15]:
len(loss_vals[0][0])

In [16]:
# Set up a grid with 3 rows and 4 columns
fig, axs = plt.subplots(5, 4, figsize=(15, 10))

# Flatten the axes array for easy iteration
axs = axs.flatten()

# Plot each loss value in the grid
for i in range(len(loss_vals)):
    axs[i].plot(np.array(loss_vals[i][0]), 'r+')
    axs[i].plot(np.array(loss_vals[i][1]), 'b.')
    axs[i].set_title(f'Plot {i+1}')

# Hide any empty subplots
for i in range(len(loss_vals), len(axs)):
    fig.delaxes(axs[i])

plt.tight_layout()
plt.show()

In [17]:
# for i in range(len(loss_vals)):
#     plt.figure(i)  # Create a new figure for each plot
#     plt.plot(np.array(loss_vals[i][0]), 'r+')
#     plt.plot(np.array(loss_vals[i][1]), 'b.')
#     plt.show()  # Display the plot

In [18]:
model_path = model_base_path/'hyperparameters.pkl'

with open(model_path, 'wb') as f:
    pickle.dump({'ncomps':ncomps, 'nevecs':nevecs,'nbasecomps':nbasecomps, 'ncauses':ncauses, 'crop_size':crop_size}, f)

In [19]:
loss_vals[0][1]